api doc : https://www.coingecko.com/api/documentations/v3

In [1]:
import numpy as np
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

In [2]:
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020'
}

session = Session()

try:
  response = session.get(url, params=parameters)
  data1 = json.loads(response.text)
  print(json.dumps(data1, indent=4, sort_keys=True))
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{
    "community_data": {
        "facebook_likes": null,
        "reddit_accounts_active_48h": "4724.23076923077",
        "reddit_average_comments_48h": 451.25,
        "reddit_average_posts_48h": 7.5,
        "reddit_subscribers": 1751352,
        "twitter_followers": 72613
    },
    "developer_data": {
        "closed_issues": 5212,
        "code_additions_deletions_4_weeks": {
            "additions": 11101,
            "deletions": -7157
        },
        "commit_count_4_weeks": 243,
        "forks": 26620,
        "pull_request_contributors": 696,
        "pull_requests_merged": 8029,
        "stars": 45884,
        "subscribers": 3590,
        "total_issues": 5791
    },
    "id": "bitcoin",
    "image": {
        "small": "https://assets.coingecko.com/coins/images/1/small/bitcoin.png?1547033579",
        "thumb": "https://assets.coingecko.com/coins/images/1/thumb/bitcoin.png?1547033579"
    },
    "localization": {
        "ar": "\u0628\u064a\u062a\u0643\u0648\u064a\u0646",


In [3]:
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020',
  'localization' : False
}

session = Session()

try:
  response = session.get(url, params=parameters)
  data1 = json.loads(response.text)
  print(json.dumps(data1, indent=4, sort_keys=True))
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


{
    "community_data": {
        "facebook_likes": null,
        "reddit_accounts_active_48h": "4724.23076923077",
        "reddit_average_comments_48h": 451.25,
        "reddit_average_posts_48h": 7.5,
        "reddit_subscribers": 1751352,
        "twitter_followers": 72613
    },
    "developer_data": {
        "closed_issues": 5212,
        "code_additions_deletions_4_weeks": {
            "additions": 11101,
            "deletions": -7157
        },
        "commit_count_4_weeks": 243,
        "forks": 26620,
        "pull_request_contributors": 696,
        "pull_requests_merged": 8029,
        "stars": 45884,
        "subscribers": 3590,
        "total_issues": 5791
    },
    "id": "bitcoin",
    "image": {
        "small": "https://assets.coingecko.com/coins/images/1/small/bitcoin.png?1547033579",
        "thumb": "https://assets.coingecko.com/coins/images/1/thumb/bitcoin.png?1547033579"
    },
    "localization": {
        "ar": "\u0628\u064a\u062a\u0643\u0648\u064a\u0646",


https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1604188800&to=1605283978
endpoint pour une fourchette de date